# Project 4

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

In [2]:
ratings = pd.read_csv(
    "ml-1m/ratings.dat", 
    sep=':',
    header=None,
    usecols=[0, 2, 4, 6],
    names=['UserID', 'MovieID', 'Rating', 'Timestamp'],
    dtype={'UserID': 'int', 'MovieID': 'int', 'Rating': 'int', 'Timestamp': 'int'}
)

ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
with open("ml-1m/movies.dat", 'r', encoding='latin1') as file:
    movies_raw = file.readlines()

movies = pd.DataFrame([line.strip().split("::") for line in movies_raw], columns=['MovieID', 'Title', 'Genres'])

movies['MovieID'] = movies['MovieID'].astype(int)
movies['Year'] = movies['Title'].str.extract(r'\((\d{4})\)').astype(int)

movies.head()

,MovieID,Title,Genres,Year
0,1,Toy Story (1995),Animation|Children's|Comedy,1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [4]:
users = pd.read_csv(
    "ml-1m/users.dat", 
    sep=':',
    header=None,
    usecols=[0, 2, 4, 6, 8],
    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    dtype={'UserID': 'int', 'Age': 'int'}
)

users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## System I: Recommendation Based on Popularity

We create a weighted ranking based on the number of 5-star and 4-star ratings, as well as the total number of ratings. It gives priority to movies with high ratings, while also accounting for overall popularity. 

The weighting score is determined by summing the following weighted components:

* <b>n_5_star</b>: Number of 5-star ratings. It carries the most weight (0.5) since it indicates high satisfaction
* <b>n_4_star</b>: Number of 4-star ratings. It has a moderate weight (0.3) as it also signifies positive feedback
* <b>n_ratings</b>: Total number of ratings. It ensures that popular movies with a broader audience also get attention

Additionally, we only consider movies that have at least 50 ratings

In [15]:
n_min = 50 # min number of ratings to be deemed relevant

movie_stats = ratings.groupby('MovieID').agg(
    RatingsCount=('Rating', 'size'), # number of ratings per movie
    AverageRating=('Rating', 'mean') # average rating per movie
).reset_index()

filtered_stats = movie_stats[movie_stats['RatingsCount'] >= n_min]

top_movies = filtered_stats.sort_values('AverageRating', ascending=False).head(10)
top_movies = pd.merge(movies, top_movies, on='MovieID')

In [16]:
n_min = 100 # min number of ratings to be deemed relevant

rating_counts = ratings.groupby(['MovieID', 'Rating']).size().unstack(fill_value=0)

rating_counts['n_5_star'] = rating_counts.get(5, 0)
rating_counts['n_4_star'] = rating_counts.get(4, 0)
rating_counts['n_ratings'] = rating_counts.sum(axis=1)

rating_counts['WeightedScore'] = (
    0.5 * rating_counts['n_5_star'] +
    0.3 * rating_counts['n_4_star'] +
    0.2 * rating_counts['n_ratings']
)

movies_with_scores = pd.merge(movies, rating_counts.reset_index(), on='MovieID')

movies_with_scores = movies_with_scores[movies_with_scores['n_ratings'] >= n_min]

top_movies = movies_with_scores.sort_values('WeightedScore', ascending=False).head(10)


images_folder = "MovieImages/"

top_movies['Image'] = top_movies['MovieID'].apply(
    lambda x: f'<img src="{images_folder}{x}.jpg" style="width:100px;height:auto;">'
)

columns_to_display = ['Image', 'MovieID', 'Title', 'WeightedScore']
display(HTML(top_movies[columns_to_display].sort_values('WeightedScore', ascending=False).to_html(escape=False, index=False)))

Image,MovieID,Title,WeightedScore
,2858,American Beauty (1999),2504.7
,260,Star Wars: Episode IV - A New Hope (1977),2274.4
,1196,Star Wars: Episode V - The Empire Strikes Back (1980),2149.6
,2028,Saving Private Ryan (1998),1941.6
,1198,Raiders of the Lost Ark (1981),1932.8
,593,"Silence of the Lambs, The (1991)",1911.6
,2571,"Matrix, The (1999)",1889.5
,2762,"Sixth Sense, The (1999)",1850.3
,1210,Star Wars: Episode VI - Return of the Jedi (1983),1845.7
,608,Fargo (1996),1795.2


In [17]:
images_folder = "MovieImages/"

top_movies['Image'] = top_movies['MovieID'].apply(
    lambda x: f'<img src="{images_folder}{x}.jpg" style="width:100px;height:auto;">'
)

columns_to_display = ['Image', 'MovieID', 'Title', 'AverageRating']
display(HTML(top_movies[columns_to_display].sort_values('AverageRating', ascending=False).to_html(escape=False, index=False)))

KeyError: "['AverageRating'] not in index"